# Introduction

This notebook will develop a machine learning model to perform fraudulent transaction detection for transactions performed on a mobile payment system. The transaction data is generated by a simulator that generates 31 days of transactions that are statistically similar to real transactions performed on a real mobile payment system for the same period of time. If you would like to know more about the data set you can go [here](https://www.kaggle.com/ntnu-testimon/paysim1).

# Module Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import pickle
import os
import datetime
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve

from xgboost import XGBClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

import pymysql.cursors
import sys

#to speed up pandas operands
from pandarallel import pandarallel

#GPU

import cupy as cnp
import cudf
import numpy as np
import pandas as pd
import datetime as dt
import gc
from sklearn.preprocessing import LabelEncoder
from cuml import train_test_split as gputrain_test_split
from cuml import LinearRegression as gpuLinearRegression
from cuml import KMeans as gpuKmeans
from cuml import LogisticRegression as gpuLogisticRegression
from cuml.ensemble import RandomForestClassifier as gpuRandomForestClassifier

In [10]:
#These are top-level variables used to control the operation of the notebook

#random seed used to initialize machine learning models and data set maniuplation functions
RANDOM_STATE = 999

#used to determine what percent of the overall data set will be used for testing trained models
TEST_SIZE = 0.1

# When this is a 1, this notebook will access clean the data on the SQL server and download the data to this notebook
# please note this will require credentials and it takes time
#
# In lieu of that I can make available a pickle file of the post-cleaned SQL table in which case
# you can ask me for it and you won't need to set this variable to 1.
#
RUN_TABLE_CREATE_IMPORT = 1

# Data Import Data Cleaning In SQL

The data is uploaded to a remote SQL server. The credentials to that server is accessesd by this notebook in a file held securely. If you would like access to this server please let me know and I'll send it to you. 

Here I am reading in the first few lines of the data set to get the names of the columns and the types of data within each column.

In [18]:
data_df = pd.read_csv("../Data/PS_20174392719_1491204439457_log.csv",nrows=5)
data_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [19]:
#COL_NAMES will be used to clean the database on the SQL server
COL_NAMES = data_df.columns
COL_NAMES = list(COL_NAMES)
COL_NAMES.remove("type")
COL_NAMES.append("transtype")
for i, a_col in enumerate(COL_NAMES):
    print(a_col)

step
amount
nameOrig
oldbalanceOrg
newbalanceOrig
nameDest
oldbalanceDest
newbalanceDest
isFraud
isFlaggedFraud
transtype


So this information will be used to create the SQL table.

Obtaining credentials to log into the SQL server. Note this was developed to work on a mac or a Linux machine.

In [12]:
#Establishing connection to mariaDB server
#Note, sever_details.txt is not provided you'll have to ask me for it
if sys.platform == 'linux':
    server_details_file_name = "/home/magreen/Dropbox/PERSONAL/Documents/Word/server_details.txt"
elif sys.platform == 'darwin':
    server_details_file_name = "/Users/magreen/Dropbox/PERSONAL/Documents/Word/server_details.txt"
else:
    print("WARNING: You are running on a system type I have not tested. Please contact me if you want help testing this.")

SERVER_DETAILS = open(server_details_file_name,"r")
line_of_text = SERVER_DETAILS.readline()

host_name, username, mypassword, db_name = line_of_text.split(',')

host_name = host_name.rstrip()
host_name = host_name.lstrip()
username = username.rstrip()
username = username.lstrip()
mypassword = mypassword.rstrip()
mypassword = mypassword.lstrip()
db_name = db_name.rstrip()
db_name = db_name.lstrip()

In [13]:
#This data frame is no longer needed so deleting it
del data_df

Creating a connection to the SQL server

In [14]:
connection = pymysql.connect(host=host_name, user=username, password=mypassword, db=db_name, cursorclass=pymysql.cursors.DictCursor)

So this code will create the table on the SQL server, upload a copy of the original [data set CSV file](https://www.kaggle.com/ntnu-testimon/paysim1), remove rows that may contain null data, and find and remove duplicate rows.

In [15]:
#get a cursor
mycursor = connection.cursor()

In [16]:
#Create table import entire CSV
if RUN_TABLE_CREATE_IMPORT != 0:
    drop_table_query = "drop table if exists paysim_data2;"

    mycursor.execute(drop_table_query)
    mycursor.execute("commit;")

    create_table_query = """


create table paysim_data2 (
    step INT,
    transtype VARCHAR(255),
    amount REAL,
    nameOrig VARCHAR(255),
    oldbalanceOrg REAL,
    newbalanceOrig REAL,
    nameDest VARCHAR(255),
    oldbalanceDest REAL,
    newbalanceDest REAL,
    isFraud INT,
    isFlaggedFraud INT
    );


    """
    mycursor.execute(create_table_query)
    mycursor.execute("commit;")

    import_query = """


load data infile '/var/lib/mysql-files/data/PS_20174392719_1491204439457_log.csv'
into table paysim_data2 fields terminated by ','  lines terminated by '\n'
ignore 1 rows;

    """

    mycursor.execute(import_query)
    mycursor.execute("commit;")
    
    #and a row ID to the table
    alter_table_query = """
    alter table paysim_data2 add column id int auto_increment primary key first;
    """
    mycursor.execute(alter_table_query)
    mycursor.execute("commit;")
    print("table creation and import done")
else:
    print("Skipping MySQL table creation and CSV import")

table creation and import done


In this section I remove rows that contain null entries or duplicate entries from the SQL table.

In [20]:
########################################################################
#
# Search each column in the MySQL table for null. If null remove the row
#
########################################################################
if RUN_TABLE_CREATE_IMPORT != 0:

    for column_name in COL_NAMES:
        temp_query = """
    
        select * from paysim_data2 where {} is null;
    
        """.format(column_name)
    
        mycursor.execute("begin;")
        query_result = mycursor.execute(temp_query)
        print("For column = {} here are the null rows:\n".format(column_name))
        print(query_result)
        print("\n")
    
        if query_result != 0:
            row_contents = mycursor.fetchall()
            print("This row contains NULL information:\n")
        
            for bad_row in row_contents:
                print(row_contents)
            
            temp_efface_row_query = """
            delete from paysim_data2 where {} is null;
            """.format(column_name)
        
            query_result = mycursor.execute(temp_efface_row_query)
        
            #print("result = {}".format(query_result))
            mycursor.execute("commit;")
            #print("result = {}".format(query_result))
            if query_result:
                print("\nsuccessfully removed row\n")
            else:
                print("error: failed to remove row!")
        
    

        
    print("\ncheck done.\n")
else:
    print("\nskipping null search and clean\n")


For column = step here are the null rows:

0


For column = amount here are the null rows:

1


This row contains NULL information:

[{'id': 6362623, 'step': 1, 'transtype': 'PAYMENT', 'amount': None, 'nameOrig': 'C3PO', 'oldbalanceOrg': 100.0, 'newbalanceOrig': 45.0, 'nameDest': 'R2D2', 'oldbalanceDest': 0.0, 'newbalanceDest': 55.0, 'isFraud': 0, 'isFlaggedFraud': 0}]

successfully removed row

For column = nameOrig here are the null rows:

0


For column = oldbalanceOrg here are the null rows:

0


For column = newbalanceOrig here are the null rows:

0


For column = nameDest here are the null rows:

0


For column = oldbalanceDest here are the null rows:

0


For column = newbalanceDest here are the null rows:

1


This row contains NULL information:

[{'id': 6362624, 'step': 1, 'transtype': 'PAYMENT', 'amount': 10.0, 'nameOrig': 'Luke', 'oldbalanceOrg': 100.0, 'newbalanceOrig': 90.0, 'nameDest': 'Wedge', 'oldbalanceDest': 0.0, 'newbalanceDest': None, 'isFraud': 0, 'isFlaggedFraud':

In [21]:
########################################################################
#
# Search for duplicate rows. Remove the second-most row for every pair of duplicate rows
#
########################################################################

search_for_duplicates_query = """
select
     step, count(step),
     transtype, count(transtype),
     amount, count(amount),
     nameOrig, count(nameOrig),
     oldbalanceOrg, count(oldbalanceOrg),
     newbalanceOrig, count(newbalanceOrig),
     nameDest, count(nameDest),
     oldbalanceDest, count(oldbalanceDest),
     newbalanceDest, count(newbalanceDest),
     isFraud, count(isFraud),
     isFlaggedFraud, count(isFlaggedFraud)
     from paysim_data2
     group by step,transtype,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
     having
     count(step) > 1 and
     count(transtype) > 1 and
     count(amount) > 1 and
     count(nameOrig) > 1 and
     count(oldbalanceOrg) > 1 and
     count(newbalanceOrig) > 1 and
     count(nameDest) > 1 and
     count(oldbalanceDest) > 1 and
     count(newbalanceDest) > 1 and
     count(isFraud) > 1 and
     count(isFlaggedFraud) > 1;



"""

remove_duplicates_query1 = """


SET SESSION old_alter_table=1;


"""

remove_duplicates_query2 = """

alter ignore table paysim_data2 add unique index u(amount,nameOrig,nameDest);


"""


remove_duplicates_query3 = """
SET SESSION old_alter_table=0;


"""
if RUN_TABLE_CREATE_IMPORT != 0:

    # removing duplicates takes too long will just search for duplicates
    # The dataset has 6M rows every duplicate remove I found 
    # does a n^2 operation which is too many for this DB
    # other methods (CTE) don't work
    mycursor.execute("begin;")
    mycursor.execute(search_for_duplicates_query)
    results = mycursor.fetchall()

    print("following are duplicate rows in the SQL DB:\n")
    for myresult in results:
        print(myresult)
    print("\ndone searching for duplicates\n")

    print("removing duplicates\n")
    mycursor.execute("begin;")
    mycursor.execute(remove_duplicates_query1)
    mycursor.execute("commit;")
    mycursor.execute(remove_duplicates_query2)
    mycursor.execute("commit;")
    mycursor.execute(remove_duplicates_query3)
    mycursor.execute("commit;")
    print("\ndone removing duplicates\n")
else:
    print("\nskipping redundant row search and remove\n")


following are duplicate rows in the SQL DB:

{'step': 1, 'count(step)': 2, 'transtype': 'PAYMENT', 'count(transtype)': 2, 'amount': 55.0, 'count(amount)': 2, 'nameOrig': 'Han Solo', 'count(nameOrig)': 2, 'oldbalanceOrg': 100.0, 'count(oldbalanceOrg)': 2, 'newbalanceOrig': 45.0, 'count(newbalanceOrig)': 2, 'nameDest': 'Princess Lea', 'count(nameDest)': 2, 'oldbalanceDest': 0.0, 'count(oldbalanceDest)': 2, 'newbalanceDest': 55.0, 'count(newbalanceDest)': 2, 'isFraud': 0, 'count(isFraud)': 2, 'isFlaggedFraud': 0, 'count(isFlaggedFraud)': 2}

done searching for duplicates

removing duplicates


done removing duplicates



# Exploratory Data Analysis

# Model Selection

## Base Line Model

### Model Evaluation

## Logistic Regression

### Model Evaluation

## Random Forest Model

### Model Evaluation


## XGBoost Model


### Model Evaluation


# Final Model Evaluation


## Precision / Recall Tradeoff

## ROC Curves and AUC Scores

# Conclusion